#### Setting up the Environment

In [ ]:
%%capture
!pip install pyyaml==5.1
!pip install sklearn funcy argparse

In [ ]:
import torch, torchvision
assert torch.__version__.startswith("1.8")

In [ ]:
# %%capture
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.9/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
import numpy as np
import os, json, cv2, random, glob, tqdm, math
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

#### Creating the Dataset

In [ ]:
from PIL import Image, ImageDraw, ImageOps
from skimage import measure
from pycocotools import mask as pymask
import json
!pip install imantics
from imantics import Polygons, Mask

In [ ]:
def get_crop_boxes(xys, boxSize, crop):
  x1,y1,x2,y2 = crop
  crop_boxes = []
  for xy in xys:
    x0, y0 = xy
    if (x1 <= x0 <= x2) and (y1 <= y0 <= y2):
      bx1, by1 = (x0 - boxSize / 2),  (y0 - boxSize / 2)
      bx2, by2 = bx1 + boxSize, by1 + boxSize
      if bx1 < x1:
        bx1 = x1
      if by1 < y1:
        by1 = y1
      if bx2 > x2:
        bx2 = x2
      if by2 > y2:
        by2 = y2
      crop_boxes.append([bx1-x1, by1 - y1, bx2 - bx1, by2 - by1]) #in x1y1wh format
  return crop_boxes

In [ ]:
def myconverter(obj):
  if isinstance(obj, np.integer):
      return int(obj)
  elif isinstance(obj, np.floating):
      return float(obj)
  elif isinstance(obj, np.ndarray):
      return obj.tolist()
  elif isinstance(obj, datetime.datetime):
      return obj.__str__()

In [ ]:
def create_crops_annos(data_dir, crop_size, dest_dir, boxSize = 30):
  all_files = os.listdir(data_dir)
  img_files = [os.path.join(data_dir,f) for f in all_files if f.endswith('.tif')]
  xy_files = [os.path.join(data_dir,f) for f in all_files if f.endswith('.txt')]
  img_files.sort()
  xy_files.sort()
  new_anno_dict = {"images":[], 
                   "annotations":[],
                   "categories": [{
                      "id": 1,
                      "name": "tassel",
                      "supercategory": "tassel",
                      "color": "#0bbdcc"
                   }]}

  crop_id = 1
  anno_id = 1
  for x in range(len(img_files)):
    img = cv2.imread(img_files[x])
    img_height, img_width = img.shape[:2]

    label_file = open(xy_files[x],'r')
    xys = [list(map(float, line.split())) for line in label_file]
    # boxes = [xy_to_xyxy(xy, 30, img_width, img_height) for xy in xys]

    for i in range(img_height // crop_size):
      for j in range(img_width // crop_size):
        x1, y1 = (crop_size * j), (crop_size * i)
        x2, y2 = (x1 + crop_size), (y1 + crop_size)
        
        crop = img[y1:y2,x1:x2]
        crop_boxes = get_crop_boxes(xys, boxSize, [x1, y1, x2, y2])
        
        crop_image = {
          "id" : crop_id,
          "category_ids": [1],
          "width": crop_size,
          "height": crop_size,
          "file_name": f'crop_{x}_{i}_{j}.jpg',
          "num_annotations": len(crop_boxes)
        }
        new_anno_dict["images"].append(crop_image)
        cv2.imwrite(f'{dest_dir}/crop_{x}_{i}_{j}.jpg', crop)

        for box in crop_boxes:
          crop_annotation = {
            "id": anno_id,
            "bbox": box,
            "image_id": crop_id,
            "segmentation": [],
            "area": box[2]*box[3],
            "iscrowd": 0,
            "category_id": 1
          }
          new_anno_dict["annotations"].append(crop_annotation)
          anno_id += 1
        crop_id += 1
  with open(f'{dest_dir}/annotations.json', 'wt', encoding='UTF-8') as anno_file:
    json.dump(new_anno_dict, anno_file, indent=2, sort_keys=True, default=myconverter)

In [ ]:
!mkdir dataset

In [ ]:
create_crops_annos('/content/drive/MyDrive/TasselNetv1/data', 150, '/content/dataset')

#### Splitting the dataset into Train Test

In [ ]:
import json
import argparse
import funcy
from sklearn.model_selection import train_test_split

def save_coco(file, images, annotations, categories):
  with open(file, 'wt', encoding='UTF-8') as coco:
    json.dump({'images': images, 
      'annotations': annotations, 'categories': categories}, coco, indent=2, sort_keys=True)
    
def filter_annotations(annotations, images):
  image_ids = funcy.lmap(lambda i: int(i['id']), images)
  return funcy.lfilter(lambda a: int(a['image_id']) in image_ids, annotations)

def split_coco_annotation(annotations, split_ratio, train_json, test_json, is_having = True):
  with open(annotations, 'rt', encoding='UTF-8') as anno:
    coco = json.load(anno)
    images = coco['images']
    annotations = coco['annotations']
    categories = coco['categories']

    images_with_annotations = funcy.lmap(lambda a: int(a['image_id']), annotations)

    if is_having:
        images = funcy.lremove(lambda i: i['id'] not in images_with_annotations, images)

    x, y = train_test_split(images, train_size = split_ratio)

    save_coco(train_json, x, filter_annotations(annotations, x), categories)
    save_coco(test_json, y, filter_annotations(annotations, y), categories)

    print("Saved {} entries in {} and {} in {}".format(len(x), train_json, len(y), test_json))

In [ ]:
split_coco_annotation('/content/dataset/annotations.json', 0.6, 
                      '/content/dataset/train_annotations.json', '/content/dataset/test_annotations.json')

#### Registering the Dataset to the Detectron and Checking the Annotations

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances(f"tassel_train", {}, '/content/dataset/train_annotations.json', "/content/dataset")
register_coco_instances(f"tassel_test", {}, '/content/dataset/test_annotations.json', "/content/dataset")

In [ ]:
tassel_metadata = MetadataCatalog.get(f"tassel_train")
train_dataset_dicts = DatasetCatalog.get(f"tassel_train")
test_dataset_dicts = DatasetCatalog.get(f"tassel_test")

In [ ]:
import random
from detectron2.utils.visualizer import ColorMode

for d in random.sample(test_dataset_dicts, 10):
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=tassel_metadata, scale=1, instance_mode=ColorMode.IMAGE_BW)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
# !unzip /content/drive/MyDrive/TasselNetv1/Maize_Tassel_Counting_Dataset.zip

In [ ]:
# import scipy.io
# mat = scipy.io.loadmat('/content/Maize Tassel Counting Dataset/Taian2012_1/Annotations/T0001_XM_20120808090256_01.mat')

In [ ]:
# mat

In [ ]:
# mat['annotation'].tolist()[0][0][1].tolist()

#### Training using Faster RCNN

In [ ]:
%%capture
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("tassel_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 10
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

#### Trying the Model on Testset


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(test_dataset_dicts, 5): 
  print(d['file_name']) 
  im = cv2.imread(d['file_name'])
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                  metadata=tassel_metadata, 
                  scale=2, 
                  instance_mode=ColorMode.IMAGE_BW
  )
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(v.get_image()[:, :, ::-1])

#### Implementation

In [ ]:
def get_tassels(img, predictor, crop_size = 150):
  num_tassels = 0
  img_height, img_width = img.shape[:2]
  tassels_img = Image.new('RGB', (img_width // crop_size * crop_size, img_height // crop_size * crop_size))
  for i in range(img_height // crop_size):
    for j in range(img_width // crop_size):
      x1, y1 = (crop_size * j), (crop_size * i)
      x2, y2 = (x1 + crop_size), (y1 + crop_size)
      crop = img[y1:y2,x1:x2]
      output = predictor(crop)
      boxes = outputs['instances'].get_fields()['pred_boxes'].tensor.tolist()
      num_tassels += len(boxes)
      tassels_crop = Visualizer(crop[:, :, ::-1],
                metadata=tassel_metadata, 
                scale=1, 
                instance_mode=ColorMode.IMAGE_BW
      )
      tassels_crop = tassels_crop.draw_instance_predictions(output["instances"].to("cpu"))
      tassels_crop = tassels_crop.get_image()[:, :, ::-1]
      tassels_img.paste(Image.fromarray(tassels_crop),(j* crop_size, i * crop_size))
  return num_tassels, np.array(tassels_img)

In [ ]:
im = cv2.imread('/content/drive/MyDrive/TasselNetv1/data/Crop2.tif')
num_tassels, tassels_img = get_tassels(im, predictor, 150)
# outputs = predictor(im)
# v = Visualizer(im[:, :, ::-1],
#                 metadata=tassel_metadata, 
#                 scale=2, 
#                 instance_mode=ColorMode.IMAGE_BW
# )
# v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
num_tassels

In [ ]:
cv2_imshow(tassels_img)

In [ ]:
!cp -R /content/output/model_final.pth /content/drive/MyDrive/TasselNetv1/model_final.pth